In [ ]:
!pip install wandb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import os

from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import datetime as dt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, TimeDistributed, \
    BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import wandb
from wandb.keras import WandbCallback


In [ ]:
random.seed(hash("random seed") % 2**32 - 1)
np.random.seed(hash("numpy seed") % 2**32 - 1)
tf.random.set_seed(hash("tensorflow seed") % 2**32 - 1)

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
def create_datasets(dataframe, split, steps, lookback, horizon, batch_size, scaler='standard'):
    # Split method
    train_split = int(split * dataframe.shape[0])
    val_split = int((split+0.1) * dataframe.shape[0])

    train = dataframe.iloc[:train_split]
    val = dataframe.iloc[train_split:val_split]
    test = dataframe.iloc[val_split:]

    # Scaler
    if scaler == 'standard':
        X_scaler = StandardScaler()
        y_scaler = StandardScaler()
    elif scaler == 'minmax':
        X_scaler = MinMaxScaler()
        y_scaler = MinMaxScaler()
    else:
        print("Please specify one of 'standard' or 'minmax' to scaler parameter.")
    
    # Training
    start = lookback + horizon
    end = start + train_split

    X_train = train.values
    y_train = dataframe.iloc[start:end][['Value']]

    X_train = X_scaler.fit_transform(X_train)
    y_train = y_scaler.fit_transform(y_train)

    # Validation
    x_end = len(val) - lookback - horizon
    y_val_start = train_split + lookback + horizon

    X_val = val.iloc[:x_end]
    y_val = dataframe.iloc[y_val_start:y_val_start+x_end][['Value']]

    X_val = X_scaler.transform(X_val)
    y_val = y_scaler.transform(y_val)

    # Test
    x_end = len(test) - lookback - horizon
    y_test_start = val_split + lookback + horizon

    X_test = test.iloc[:x_end]
    y_test = dataframe.iloc[y_test_start:y_test_start+x_end][['Value']]

    X_test = X_scaler.transform(X_test)
    y_test = y_scaler.transform(y_test)

    # Batch Sequence Generators
    sequence_length = int(lookback/steps)

    dataset_train = timeseries_dataset_from_array(
        X_train, y_train,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )

    dataset_val = timeseries_dataset_from_array(
        X_val, y_val,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )    

    dataset_test = timeseries_dataset_from_array(
        X_test, y_test,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )    

    return dataset_train, dataset_val, dataset_test

def ts_offset_split(dataframe, steps, lookback, horizon, batch_size, scaler='standard'):
    start = 0
    end = 168
    offset = 24
    training = []
    validation = []

    for i in range(int((365+366)/8)):

        train = dataframe.iloc[start:end]
        val = dataframe.iloc[end:end+offset]
        training.append(train)
        validation.append(val)

        start += 192
        end += 192
    
    # Decide Splits for sets
    train = pd.concat(training)

    val = pd.concat(validation)
    train = train.append(dataframe[(dataframe.index.date > val.index.max()) & (dataframe.index.date < dt.date(2021,1,1))])

    test = dataframe[dataframe.index.date >= dt.date(2021,1,1)]
    
    tmpdf = pd.concat([train,val,test])

    # Scaler
    if scaler == 'standard':
        X_scaler = StandardScaler()
        y_scaler = StandardScaler()
    elif scaler == 'minmax':
        X_scaler = MinMaxScaler()
        y_scaler = MinMaxScaler()
    else:
        print("Please specify one of 'standard' or 'minmax' to scaler parameter.")
    
    # Training
    start = lookback + horizon
    end = start + train.shape[0]

    X_train = train.values
    y_train = tmpdf.iloc[start:end][['Value']]

    X_train = X_scaler.fit_transform(X_train)
    y_train = y_scaler.fit_transform(y_train)

    # Validation
    x_end = len(val) - lookback - horizon
    y_val_start = train.shape[0] + lookback + horizon

    X_val = val.iloc[:x_end]
    y_val = tmpdf.iloc[y_val_start:y_val_start+x_end][['Value']]

    X_val = X_scaler.transform(X_val)
    y_val = y_scaler.transform(y_val)

    # Test
    x_end = len(test) - lookback - horizon
    y_test_start = (train.shape[0] + val.shape[0]) + lookback + horizon

    X_test = test.iloc[:x_end]
    y_test = tmpdf.iloc[y_test_start:y_test_start+x_end][['Value']]

    X_test = X_scaler.transform(X_test)
    y_test = y_scaler.transform(y_test)

    # Batch Sequence Generators
    sequence_length = int(lookback/steps)

    dataset_train = timeseries_dataset_from_array(
        X_train, y_train,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )

    dataset_val = timeseries_dataset_from_array(
        X_val, y_val,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )    

    dataset_test = timeseries_dataset_from_array(
        X_test, y_test,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )    

    return dataset_train, dataset_val, dataset_test

In [ ]:
def load_data(config, splitmethod):
    # Load csv & parse dates to datetime index
    data = pd.read_csv(f'/content/drive/MyDrive/Thesis/{config.dataset}_features.csv', index_col='Datetime', parse_dates=['Datetime'])
    # Select Features
    data = data[['Value', 'sunshine_mins', 'airtemp_c', 'daylength_hrs', 'wkdy_sin', 'wkdy_cos', 'wknd', 'mnth_sin', 'mnth_cos']]
    
    if splitmethod == 'offset':
        train, val, test = ts_offset_split(data, 
                                  steps=steps, lookback=config.lookback, 
                                  horizon=horizon, batch_size=config.batch_size, 
                                  scaler='standard')
    
    elif splitmethod == 'standard':
        train, val, test = create_datasets(data, split=config.splitrate, 
                                  steps=steps, lookback=config.lookback, 
                                  horizon=horizon, batch_size=config.batch_size, 
                                  scaler='standard')
    
    return train, val, test
    
    
def build_model(config):
        
    model = Sequential()

    model.add(Input(shape=(config.lookback, config.num_features)))

    # CNN Block
    model.add(Conv1D(filters=config.cnn_layer_size_1, kernel_size=config.kernelsize, activation=config.activation_cnn))
    model.add(MaxPooling1D(pool_size=2))
    
    # RNN Block  
    model.add(GRU(config.gru_layer_size_1, return_sequences=False, activation=config.activation_gru))
    model.add(Dropout(config.dropout))

    model.add(Dense(1))
    
    opt = config.optimizer
    if opt == 'sgd':
        optimizer = SGD(learning_rate=config.learning_rate, momentum=config.momentum)
        
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

def log_defaults():
    # Default values from the first sweep
    # The following values resulted in the strongest model
    wandb_config = {
        'num_features' : 9,
        'epochs' : 100,
        'batch_size' : 222,
        'optimizer' : 'sgd',
        'dropout' : 0.2,
        'lookback' : 744,
        'activation_cnn' : 'relu',
        'activation_gru' : 'tanh',
        'kernelsize': 7,
        'cnn_layer_size_1' : 179,
        'gru_layer_size_1' : 95,
        'learning_rate' : 0.04,
        'momentum' : 0.9,
        'dataset' : 'kolding',
        'splitrate': 0.8,
        'splitmethod': 'offset',
        
    }
    return wandb_config

def run_tuner():
    
    wandb.init(config=log_defaults(), group='cnnrnn-sweep-grid-offset-2', project='thesis')
    
    model = build_model(config=wandb.config)
    
    train, val, test = load_data(config=wandb.config, splitmethod=wandb.config.splitmethod)
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=5)

    callbacks = [WandbCallback(), reduce_lr]
    
    model.fit(
        train,
        epochs=wandb.config.epochs,
        validation_data=val,
        callbacks=callbacks
    )

    eval = model.evaluate(test)

    wandb.log({'test_loss': eval[0]})
    wandb.log({'test_mae': eval[1]})


In [ ]:
# Sweep Config
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 5
    },
    'parameters': {
        'num_features' : {
            'value' : 9
        },
        'batch_size': {
            'values': [64, 224]
        },
        'lookback': {
            'value': 24*31
        },
        'optimizer': {
            'value': 'sgd'
        },
        'dropout': {
            'value': 0.2
        },
        'epochs': {
            'value': 30
        },
        'activation_gru': {
            'value': 'tanh'
        },
        'activation_cnn': {
            'value': 'relu'
        },
        'kernelsize': {
            'values': [3, 7]
        },
        'cnn_layer_size_1': {
            'values': [56, 176]
        },
        'gru_layer_size_1': {
            'values': [96, 192]
        },
        'learning_rate': {
            'value': 0.04
        },
        'momentum': {
            'value': 0.9
        },
        'dataset': {
            'values': ['kolding', 'strib']
        },
        'splitrate': {
            'value': 0.8
        },
        'splitmethod': {
            'value': 'offset'
        }
    }
}

In [ ]:
# Additional Parameters
steps = 1 # timesteps: 1 hour
horizon = 1 # the target hour in the future we want to predict 1 hour ahead



In [ ]:
sweep_id = wandb.sweep(sweep_config, project='thesis')

Create sweep with ID: 0xw7ge82
Sweep URL: https://wandb.ai/nbvanting/thesis/sweeps/0xw7ge82


In [ ]:
wandb.agent(sweep_id, function=run_tuner)

wandb: Agent Starting Run: wv28k353 with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 64
wandb: 	cnn_layer_size_1: 56
wandb: 	dataset: kolding
wandb: 	dropout: 0.2
wandb: 	epochs: 30
wandb: 	gru_layer_size_1: 96
wandb: 	kernelsize: 3
wandb: 	learning_rate: 0.04
wandb: 	lookback: 744
wandb: 	momentum: 0.9
wandb: 	num_features: 9
wandb: 	optimizer: sgd
wandb: 	splitmethod: offset
wandb: 	splitrate: 0.8
wandb: Currently logged in as: nbvanting (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/30
229/229 [==============================] - 54s 102ms/step - loss: 0.5446 - mae: 0.5516 - val_loss: 0.3377 - val_mae: 0.4311
Epoch 2/30
229/229 [==============================] - 22s 93ms/step - loss: 0.3089 - mae: 0.4182 - val_loss: 0.2045 - val_mae: 0.3430
Epoch 3/30
229/229 [==============================] - 21s 93ms/step - loss: 0.1856 - mae: 0.3335 - val_loss: 0.1561 - val_mae: 0.3054
Epoch 4/30
229/229 [==============================] - 22s 96ms/step - loss: 0.1531 - mae: 0.3010 - val_loss: 0.1381 - val_mae: 0.2902
Epoch 5/30
229/229 [==============================] - 22s 97ms/step - loss: 0.1307 - mae: 0.2789 - val_loss: 0.1377 - val_mae: 0.2920
Epoch 6/30
229/229 [==============================] - 21s 93ms/step - loss: 0.1221 - mae: 0.2694 - val_loss: 0.1298 - val_mae: 0.2842
Epoch 7/30
229/229 [==============================] - 22s 95ms/step - loss: 0.1227 - mae: 0.2695 - val_loss: 0.1149 - val_mae: 0.2646
Epoch 8/30
229/229 [==============================] - 22s 95m

epoch,29
loss,0.06213
mae,0.19244
val_loss,0.08825
val_mae,0.22897
_runtime,696
_timestamp,1620647636
_step,31
best_val_loss,0.08825
best_epoch,29
test_loss,0.13285


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▅▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▆▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▂▂▂▂▂▂▂▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁
val_mae,█▅▄▃▃▃▂▃▂▂▁▁▂▂▁▂▁▁▁▁▁▁▁▁▂▁▂▂▁▁
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇█████
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇█████
_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
test_loss,▁
test_mae,▁


Run wv28k353 errored: IndexError('list index out of range')
wandb: ERROR Run wv28k353 errored: IndexError('list index out of range')
wandb: Agent Starting Run: rrxnc7sz with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 64
wandb: 	cnn_layer_size_1: 56
wandb: 	dataset: kolding
wandb: 	dropout: 0.2
wandb: 	epochs: 30
wandb: 	gru_layer_size_1: 96
wandb: 	kernelsize: 7
wandb: 	learning_rate: 0.04
wandb: 	lookback: 744
wandb: 	momentum: 0.9
wandb: 	num_features: 9
wandb: 	optimizer: sgd
wandb: 	splitmethod: offset
wandb: 	splitrate: 0.8
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

Run qbcuo68s errored: IndexError('list index out of range')
wandb: ERROR Run qbcuo68s errored: IndexError('list index out of range')
wandb: Agent Starting Run: 6aq2edwm with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 64
wandb: 	cnn_layer_size_1: 56
wandb: 	dataset: strib
wandb: 	dropout: 0.2
wandb: 	epochs: 30
wandb: 	gru_layer_size_1: 96
wandb: 	kernelsize: 7
wandb: 	learning_rate: 0.04
wandb: 	lookback: 744
wandb: 	momentum: 0.9
wandb: 	num_features: 9
wandb: 	optimizer: sgd
wandb: 	splitmethod: offset
wandb: 	splitrate: 0.8
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/30
229/229 [==============================] - 25s 100ms/step - loss: 0.4797 - mae: 0.5133 - val_loss: 0.2097 - val_mae: 0.3536
Epoch 2/30
229/229 [==============================] - 23s 99ms/step - loss: 0.1818 - mae: 0.3235 - val_loss: 0.1535 - val_mae: 0.2915
Epoch 3/30
229/229 [==============================] - 22s 97ms/step - loss: 0.1369 - mae: 0.2801 - val_loss: 0.1459 - val_mae: 0.2804
Epoch 4/30
229/229 [==============================] - 23s 98ms/step - loss: 0.1168 - mae: 0.2632 - val_loss: 0.1185 - val_mae: 0.2491
Epoch 5/30
229/229 [==============================] - 21s 93ms/step - loss: 0.1041 - mae: 0.2477 - val_loss: 0.1260 - val_mae: 0.2640
Epoch 6/30
229/229 [==============================] - 22s 98ms/step - loss: 0.0927 - mae: 0.2328 - val_loss: 0.1252 - val_mae: 0.2680
Epoch 7/30
229/229 [==============================] - 22s 96ms/step - loss: 0.0915 - mae: 0.2316 - val_loss: 0.1206 - val_mae: 0.2559
Epoch 8/30
229/229 [==============================] - 21s 93m

epoch,29
loss,0.0595
mae,0.18639
val_loss,0.0908
val_mae,0.21541
_runtime,677
_timestamp,1620651044
_step,31
best_val_loss,0.08994
best_epoch,24
test_loss,0.1118


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▄▃▃▃▃▃▂▂▂▂▂▃▂▃▁▁▁▁▁▁▁▂▁▁▁▂▁▁
val_mae,█▅▄▃▃▄▃▃▂▂▂▃▃▃▂▃▁▁▂▁▁▁▁▂▁▁▁▂▁▁
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇█████
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇█████
_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
test_loss,▁
test_mae,▁


Run 6aq2edwm errored: IndexError('list index out of range')
wandb: ERROR Run 6aq2edwm errored: IndexError('list index out of range')
Detected 5 failed runs in a row at start, killing sweep.
wandb: ERROR Detected 5 failed runs in a row at start, killing sweep.
wandb: To change this value set WANDB_AGENT_MAX_INITIAL_FAILURES=val
